In [2]:
import numpy as np
import pandas as pd
import pennylane as qml
import matplotlib.pyplot as plt
import strawberryfields as sf

from strawberryfields.ops import *
from pennylane.templates import QAOAEmbedding
import flatnetwork_simple as fns

# Working Circuit Embedding

In [39]:
wires = 2
#Setup
K = 20
dt = 1.0

#Parameters
mu = lambda t,i: 1.0
rho = 1.0

#Annealing params
J = 1.0
theta = -J*dt/K

L = [3,1,1] 
d = 2
nmodes = int(np.prod(L))

fnet = fns.FlatNetwork(L,d)
fnet.loadParams(mu, rho)
fnet.make_mpos()

S, D = fnet.getHamiltonian()

layer1   = [S[i] for i in S] # only works when len(layer1)
layer2   = [D[i] for i in D] # and len(layer2) are equal
weights  = [layer1, layer2]

dev = qml.device('default.qubit', wires=wires)

@qml.qnode(dev)
def circuit(weights, f=None):
    QAOAEmbedding(features=f, weights=weights, wires=range(wires))
    #QAOAEmbedding(features=f, weights=weights, wires=range(2), local_field='Z')
    return qml.expval(qml.PauliZ(0))

print(S, D)
print(circuit(weights, f=features)) #outputs a measurement of an expected value of PauliZ matrix
print(circuit.draw())

Max MPO dim:  2
{(0, 0): -1.5, (1, 1): -1.5, (2, 2): -1.5} {(0, 0): 0.25, (1, 1): 0.25, (2, 2): 0.25}
0.30599237067365803
 0: ──H──╭RZ(-1.5)──RY(-1.5)──H──╭RZ(0.25)──RY(0.25)──H──┤ ⟨Z⟩ 
 1: ──H──╰RZ(-1.5)──RY(-1.5)──H──╰RZ(0.25)──RY(0.25)──H──┤     



# Custom Circuit Definition

In [40]:
K = 20
dt = 1.0

#Parameters
mu = lambda t,i: 1.0
rho = 1.0

#Annealing params
J = 1.0
theta = -J*dt/K

L = [2,2,2] 
d = 3
nmodes = int(np.prod(L))

fnet = fns.FlatNetwork(L,d)
fnet.loadParams(mu, rho)
fnet.make_mpos()

S, D = fnet.getHamiltonian()

#S, D

Max MPO dim:  9


In [160]:
def mixer_hamiltonian(num_qubits, coeffs = []):
    if len(coeffs) < 1:
        coeffs = [1] * num_qubits
    x_gates = [qml.PauliX(i) for i in range(num_qubits)]
    return qml.Hamiltonian(coeffs, x_gates)

print(mixer_hamiltonian(10))

  (1) [X0]
+ (1) [X1]
+ (1) [X2]
+ (1) [X3]
+ (1) [X4]
+ (1) [X5]
+ (1) [X6]
+ (1) [X7]
+ (1) [X8]
+ (1) [X9]


In [161]:
def singles_hamiltonian(num_qubits, coeffs = []):
    coeffs  = [1] * num_qubits
    z_gates = [qml.PauliZ(i) for i in range(num_qubits)]
    return qml.Hamiltonian(coeffs, z_gates)

print(singles_hamiltonian(10))

  (1) [Z0]
+ (1) [Z1]
+ (1) [Z2]
+ (1) [Z3]
+ (1) [Z4]
+ (1) [Z5]
+ (1) [Z6]
+ (1) [Z7]
+ (1) [Z8]
+ (1) [Z9]


In [166]:
D.keys()

dict_keys([(0, 0), (0, 1), (0, 3), (0, 2), (1, 1), (1, 3), (2, 2), (2, 3), (3, 3), (4, 4), (4, 5), (4, 7), (4, 6), (5, 5), (5, 7), (6, 6), (6, 7), (7, 7)])

In [ ]:
def double_hamiltonian(num_qubits, coeffs = []):
    coeffs